Import  required packages

In [1]:
!pip install gensim
!pip install xgboost
!pip install catboost
!pip install lightgbm
!pip install seaborn
!pip install statsmodels
!pip install pmdarima
!pip install vecstack
!pip install lightgbm
!pip install hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19861 sha256=4f406d5681a5277f27fe6802d7ea5ab96f7534e098c77f427f1f027a2f7322c7
  Stored in directory: /root/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack


In [2]:
import pandas as pd
import numpy as np

# from tqdm import tqdm
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
# from lightgbm import LGBMRegressor
# from statsmodels.tsa.arima.model import ARIMA
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split

# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Change column display number during print
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

In [3]:
# Read features files

train_df_path = '/content/drive/MyDrive/Fossil/data/Feature-Engineering-Train.csv'
val_df_path = '/content/drive/MyDrive/Fossil/data/Feature-Engineering-Test.csv'

train_df = pd.read_csv(train_df_path)
val_df = pd.read_csv(val_df_path)

In [4]:
train_df.shape, val_df.shape

((43018, 118), (1889, 118))

In [5]:
train_df.head()

,sku_name,starting_inventory,sellin,sellin_channel_1,sellin_channel_2,sellin_channel_3,sellin_channel_4,sellin_channel_5,sellin_channel_6,sellin_channel_7,sellin_channel_8,sellout,onhand_inventory,leftover_inventory,sellout_channel_1,sellout_channel_2,sellout_channel_3,sellout_channel_4,sellout_channel_5,sellout_channel_6,sellout_channel_7,sellout_channel_8,sellout_channel_9,sellout_channel_10,onhand_inventory_channel_1,onhand_inventory_channel_2,onhand_inventory_channel_3,onhand_inventory_channel_4,onhand_inventory_channel_5,onhand_inventory_channel_6,onhand_inventory_channel_7,onhand_inventory_channel_8,onhand_inventory_channel_9,onhand_inventory_channel_10,price,month,year,product_lifecycle_stage,FLAG100,disc_month,cum_disc,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN,Weeks,product_sku_embedding,season,starting_inventory_lag_3,sellin_channel_1_lag_3,sellin_channel_2_lag_3,sellin_channel_3_lag_3,sellin_channel_4_lag_3,sellin_channel_5_lag_3,sellin_channel_6_lag_3,sellin_channel_7_lag_3,sellin_channel_8_lag_3,sellout_lag_3,onhand_inventory_lag_3,leftover_inventory_lag_3,sellout_channel_1_lag_3,sellout_channel_2_lag_3,sellout_channel_3_lag_3,sellout_channel_4_lag_3,sellout_channel_5_lag_3,sellout_channel_6_lag_3,sellout_channel_7_lag_3,sellout_channel_8_lag_3,sellout_channel_9_lag_3,sellout_channel_10_lag_3,onhand_inventory_channel_1_lag_3,onhand_inventory_channel_2_lag_3,onhand_inventory_channel_3_lag_3,onhand_inventory_channel_4_lag_3,onhand_inventory_channel_5_lag_3,onhand_inventory_channel_6_lag_3,onhand_inventory_channel_7_lag_3,onhand_inventory_channel_8_lag_3,onhand_inventory_channel_9_lag_3,onhand_inventory_channel_10_lag_3,price_lag_3,starting_inventory_lead_12,sellin_channel_1_lead_12,sellin_channel_2_lead_12,sellin_channel_3_lead_12,sellin_channel_4_lead_12,sellin_channel_5_lead_12,sellin_channel_6_lead_12,sellin_channel_7_lead_12,sellin_channel_8_lead_12,sellout_lead_12,onhand_inventory_lead_12,leftover_inventory_lead_12,sellout_channel_1_lead_12,sellout_channel_2_lead_12,sellout_channel_3_lead_12,sellout_channel_4_lead_12,sellout_channel_5_lead_12,sellout_channel_6_lead_12,sellout_channel_7_lead_12,sellout_channel_8_lead_12,sellout_channel_9_lead_12,sellout_channel_10_lead_12,onhand_inventory_channel_1_lead_12,onhand_inventory_channel_2_lead_12,onhand_inventory_channel_3_lead_12,onhand_inventory_channel_4_lead_12,onhand_inventory_channel_5_lead_12,onhand_inventory_channel_6_lead_12,onhand_inventory_channel_7_lead_12,onhand_inventory_channel_8_lead_12,onhand_inventory_channel_9_lead_12,onhand_inventory_channel_10_lead_12,price_lead_12,YoY_growth,ratio_inventory,Is_backorder,avg_sellout_by_season,avg_onhand_inventory_by_season
0,YOSHTLYNYOSH,0,32416,4052,0,23299,0,0,5065,0,0,28364,572345,4052,0,0,22286,0,2026,3039,0,0,0,1013,0,0,515617,5065,37481,0,0,0,0,14182,245,1,2016,1,0.0,0,0,0,1,0,1,1.259432,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.250000,0,46598.0,239068.0
1,YOSHRTHATRAN,0,346446,15195,0,300861,4052,21273,4052,0,0,9117,337329,337329,0,0,2026,0,3039,4052,0,0,0,0,6078,0,298835,4052,16208,0,0,0,3039,9117,105,1,2016,1,0.0,0,0,0,0,1,1,0.308167,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0,46598.0,239068.0
2,YOSHOVANTERR,0,1013,0,0,0,1013,0,0,0,0,0,1013,1013,0,0,0,0,0,0,0,0,0,0,0,0,0,1013,0,0,0,0,0,0,115,1,2016,1,1.0,0,0,0,0,1,1,0.597582,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [6]:
# Data cleaning

train_df.fillna(0,inplace=True)
val_df.fillna(0,inplace=True)

rows_negative_vlaues = train_df[train_df["sellin"]<0]
train_df.drop(rows_negative_vlaues.index, axis=0, inplace = True)

rows_negative_vlaues = val_df[val_df["sellin"]<0]
val_df.drop(rows_negative_vlaues.index, axis=0, inplace = True)

# Replace 'inf' values in the 'YoY' feature with a reasonable value, such as the mean of the feature
train_df['YoY_growth'] = np.where(np.isinf(train_df['YoY_growth']),0, train_df['YoY_growth'])
val_df['YoY_growth'] = np.where(np.isinf(val_df['YoY_growth']),0, val_df['YoY_growth'])

In [7]:
train_df.head()

,sku_name,starting_inventory,sellin,sellin_channel_1,sellin_channel_2,sellin_channel_3,sellin_channel_4,sellin_channel_5,sellin_channel_6,sellin_channel_7,sellin_channel_8,sellout,onhand_inventory,leftover_inventory,sellout_channel_1,sellout_channel_2,sellout_channel_3,sellout_channel_4,sellout_channel_5,sellout_channel_6,sellout_channel_7,sellout_channel_8,sellout_channel_9,sellout_channel_10,onhand_inventory_channel_1,onhand_inventory_channel_2,onhand_inventory_channel_3,onhand_inventory_channel_4,onhand_inventory_channel_5,onhand_inventory_channel_6,onhand_inventory_channel_7,onhand_inventory_channel_8,onhand_inventory_channel_9,onhand_inventory_channel_10,price,month,year,product_lifecycle_stage,FLAG100,disc_month,cum_disc,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN,Weeks,product_sku_embedding,season,starting_inventory_lag_3,sellin_channel_1_lag_3,sellin_channel_2_lag_3,sellin_channel_3_lag_3,sellin_channel_4_lag_3,sellin_channel_5_lag_3,sellin_channel_6_lag_3,sellin_channel_7_lag_3,sellin_channel_8_lag_3,sellout_lag_3,onhand_inventory_lag_3,leftover_inventory_lag_3,sellout_channel_1_lag_3,sellout_channel_2_lag_3,sellout_channel_3_lag_3,sellout_channel_4_lag_3,sellout_channel_5_lag_3,sellout_channel_6_lag_3,sellout_channel_7_lag_3,sellout_channel_8_lag_3,sellout_channel_9_lag_3,sellout_channel_10_lag_3,onhand_inventory_channel_1_lag_3,onhand_inventory_channel_2_lag_3,onhand_inventory_channel_3_lag_3,onhand_inventory_channel_4_lag_3,onhand_inventory_channel_5_lag_3,onhand_inventory_channel_6_lag_3,onhand_inventory_channel_7_lag_3,onhand_inventory_channel_8_lag_3,onhand_inventory_channel_9_lag_3,onhand_inventory_channel_10_lag_3,price_lag_3,starting_inventory_lead_12,sellin_channel_1_lead_12,sellin_channel_2_lead_12,sellin_channel_3_lead_12,sellin_channel_4_lead_12,sellin_channel_5_lead_12,sellin_channel_6_lead_12,sellin_channel_7_lead_12,sellin_channel_8_lead_12,sellout_lead_12,onhand_inventory_lead_12,leftover_inventory_lead_12,sellout_channel_1_lead_12,sellout_channel_2_lead_12,sellout_channel_3_lead_12,sellout_channel_4_lead_12,sellout_channel_5_lead_12,sellout_channel_6_lead_12,sellout_channel_7_lead_12,sellout_channel_8_lead_12,sellout_channel_9_lead_12,sellout_channel_10_lead_12,onhand_inventory_channel_1_lead_12,onhand_inventory_channel_2_lead_12,onhand_inventory_channel_3_lead_12,onhand_inventory_channel_4_lead_12,onhand_inventory_channel_5_lead_12,onhand_inventory_channel_6_lead_12,onhand_inventory_channel_7_lead_12,onhand_inventory_channel_8_lead_12,onhand_inventory_channel_9_lead_12,onhand_inventory_channel_10_lead_12,price_lead_12,YoY_growth,ratio_inventory,Is_backorder,avg_sellout_by_season,avg_onhand_inventory_by_season
0,YOSHTLYNYOSH,0,32416,4052,0,23299,0,0,5065,0,0,28364,572345,4052,0,0,22286,0,2026,3039,0,0,0,1013,0,0,515617,5065,37481,0,0,0,0,14182,245,1,2016,1,0.0,0,0,0,1,0,1,1.259432,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.250000,0,46598.0,239068.0
1,YOSHRTHATRAN,0,346446,15195,0,300861,4052,21273,4052,0,0,9117,337329,337329,0,0,2026,0,3039,4052,0,0,0,0,6078,0,298835,4052,16208,0,0,0,3039,9117,105,1,2016,1,0.0,0,0,0,0,1,1,0.308167,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0,46598.0,239068.0
2,YOSHOVANTERR,0,1013,0,0,0,1013,0,0,0,0,0,1013,1013,0,0,0,0,0,0,0,0,0,0,0,0,0,1013,0,0,0,0,0,0,115,1,2016,1,1.0,0,0,0,0,1,1,0.597582,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [8]:
features = ['sku_name', 'leftover_inventory', 'sellin_channel_1', 'sellout', 'sellin_channel_4', 'sellin_channel_6', 'sellin_channel_8', 'sellin_channel_2', 'sellin_channel_3', 'sellout_channel_10', 'sellin_channel_5', 'ratio_inventory', 'sellin_channel_7', 'onhand_inventory_channel_10_lag_3', 'sellout_channel_6', 'sellout_channel_4', 'sellout_channel_3', 'onhand_inventory_channel_10', 'onhand_inventory', 'sellout_channel_10_lag_3', 'onhand_inventory_channel_9', 'leftover_inventory_lag_3', 'sellout_channel_1', 'year', 'month', 'sellout_channel_9', 'onhand_inventory_channel_2', 'sellout_channel_5', 'onhand_inventory_lag_3', 'sellin_channel_1_lag_3', 'product_lifecycle_stage', 'product_sku_embedding', 'onhand_inventory_channel_4', 'sellin']
print(features)

['sku_name', 'leftover_inventory', 'sellin_channel_1', 'sellout', 'sellin_channel_4', 'sellin_channel_6', 'sellin_channel_8', 'sellin_channel_2', 'sellin_channel_3', 'sellout_channel_10', 'sellin_channel_5', 'ratio_inventory', 'sellin_channel_7', 'onhand_inventory_channel_10_lag_3', 'sellout_channel_6', 'sellout_channel_4', 'sellout_channel_3', 'onhand_inventory_channel_10', 'onhand_inventory', 'sellout_channel_10_lag_3', 'onhand_inventory_channel_9', 'leftover_inventory_lag_3', 'sellout_channel_1', 'year', 'month', 'sellout_channel_9', 'onhand_inventory_channel_2', 'sellout_channel_5', 'onhand_inventory_lag_3', 'sellin_channel_1_lag_3', 'product_lifecycle_stage', 'product_sku_embedding', 'onhand_inventory_channel_4', 'sellin']


In [9]:
train_df = train_df[features]
val_df = val_df[features]

In [10]:
train_df.shape, val_df.shape

((43017, 34), (1887, 34))

In [ ]:
# Adding Arima Forecast feature to the data

from pmdarima import auto_arima

def generate_arima_forecasts(train_df, target_column="sellout", steps_ahead=4):
    """Generate ARIMA-based forecasts for each SKU (time series)."""
    base_forecasts = []  # Store forecasts for each row
    unique_skus = train_df['sku_name'].unique()

    for sku in tqdm(unique_skus, desc="Generating ARIMA forecasts"):
        sku_data = train_df[train_df['sku_name'] == sku]
        time_series = sku_data[target_column].values

        # Handle short or constant time series
        if len(time_series) < 10 or np.all(time_series == time_series[0]):
            forecast = [time_series[-1]] * min(len(sku_data), steps_ahead)  # Naive forecast
        else:
            try:
                # Auto ARIMA for robust order selection
                arima_model = auto_arima(
                    time_series,
                    seasonal=False,
                    error_action='ignore',
                    suppress_warnings=True,
                    stepwise=True
                )
                forecast = arima_model.predict(n_periods=min(len(sku_data), steps_ahead))
            except Exception as e:
                print(f"ARIMA failed for SKU {sku}: {e}")
                forecast = [np.nan] * len(sku_data)  # Fallback to NaN forecasts

        # Expand forecasts to match the original data length for this SKU
        forecast = np.pad(forecast, (0, len(sku_data) - len(forecast)), constant_values=0)
        base_forecasts.extend(forecast)

    # Validate forecast length
    if len(base_forecasts) != len(train_df):
        raise ValueError(f"Forecast length ({len(base_forecasts)}) does not match DataFrame length ({len(train_df)})")

    # Add ARIMA forecasts to the original DataFrame
    train_df["arima_forecast"] = pd.Series(base_forecasts, dtype="float64")
    return train_df

train_df = generate_arima_forecasts(train_df, target_column="sellin", steps_ahead=3)
val_df = generate_arima_forecasts(val_df, target_column="sellin", steps_ahead=3)

train_df.to_csv(arima_features_train,index=False)
val_df.to_csv(arima_features_val,index=False)

In [11]:
arima_features_train = '/content/drive/MyDrive/Fossil/data/arima-features.csv'
arima_features_val = '/content/drive/MyDrive/Fossil/data/arima-features_val.csv'

In [ ]:
# Read final set of features

In [12]:
train_df = pd.read_csv(arima_features_train)
val_df = pd.read_csv(arima_features_val)

train_df['arima_forecast'].fillna(0,inplace=True)
val_df["arima_forecast"].fillna(0,inplace=True)

In [13]:
if 'sku_name' in train_df.columns:
  train_df.drop('sku_name',axis=1,inplace=True)
if 'sku_name' in val_df.columns:
  val_df.drop('sku_name',axis=1,inplace=True)

## Hyper paramter tuning using Bayesian optimisation

LGBMRegressor

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from lightgbm import LGBMRegressor


# Assume train_df is your DataFrame and it's sorted chronologically
split_index = int(len(train_df) * 0.8)  # Adjust this to change the train/validation ratio

# Features and target extraction based on the split
X_train_new = np.array(train_df.drop('sellin', axis=1).iloc[:split_index])
y_train_new = np.array(np.log1p(train_df['sellin'].iloc[:split_index]))

X_val = np.array(train_df.drop('sellin', axis=1).iloc[split_index:])
y_val = np.array(np.log1p(train_df['sellin'].iloc[split_index:]))

from hyperopt import hp
from hyperopt import STATUS_OK
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, Trials


space = {
    'max_depth': hp.choice('max_depth', np.arange(5, 16, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'num_leaves': hp.choice('num_leaves', np.arange(20, 150, dtype=int)),
    'min_child_weight': hp.uniform('min_child_weight', 0.01, 10),
}

def objective(space):
    model = LGBMRegressor(
        max_depth=int(space['max_depth']),
        learning_rate=space['learning_rate'],
        num_leaves=int(space['num_leaves']),
        min_child_weight=space['min_child_weight'],
        n_estimators=2500
    )

    model.fit(X_train_new, y_train_new)
    pred = model.predict(X_val)

    mse = mean_squared_error(y_val, pred)
    return {'loss': mse, 'status': STATUS_OK}



trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=25,  # Number of evaluations can be adjusted based on computational resources
            trials=trials)

print("Best hyperparameters: ", best)



Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

CatBoostRegressor

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from catboost import CatBoostRegressor

space = {
    'depth': hp.choice('depth', np.arange(4, 11, dtype=int)),  # Typical ranges for CatBoost
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),  # Using log scale for learning rate
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),  # Regularization term
    'border_count': hp.choice('border_count', [32, 64, 128, 256]),
}

def objective(space):
    model = CatBoostRegressor(
        depth=int(space['depth']),
        learning_rate=space['learning_rate'],
        n_estimators=1500,
        l2_leaf_reg=space['l2_leaf_reg'],
        border_count=int(space['border_count']),
        verbose=0
    )

    model.fit(X_train_new, y_train_new)
    pred = model.predict(X_val)

    mse = mean_squared_error(y_val, pred)
    return {'loss': mse, 'status': STATUS_OK}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=25,  # Adjust based on computational resources
            trials=trials)

print("Best hyperparameters: ", best)


100%|██████████| 25/25 [10:51<00:00, 26.05s/trial, best loss: 0.044998309246028094]
Best hyperparameters:  {'border_count': 3, 'depth': 2, 'l2_leaf_reg': 8.010730871768292, 'learning_rate': 0.09969806104717538}


XGBRegressor

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from xgboost import XGBRegressor

space = {
    'max_depth': hp.choice('max_depth', np.arange(3, 10, 1, dtype=int)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'n_estimators': 1000,
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2))
}



def objective(params):
    # Ensure parameters that need to be integers are integers
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])

    # Train the model with given parameters
    model = XGBRegressor(**params)
    model.fit(X_train_new, y_train_new)

    # Predict and calculate MSE
    pred = model.predict(X_val)
    mse = mean_squared_error(y_val, pred)
    return {'loss': mse, 'status': STATUS_OK}


# Set up trials to store information
trials = Trials()

# Run optimization
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=25,  # Adjust number based on desired thoroughness and available computational resources
            trials=trials)

print("Best hyperparameters:", best)



100%|██████████| 25/25 [04:11<00:00, 10.06s/trial, best loss: 0.031697466098383226]
Best hyperparameters: {'colsample_bytree': 0.8031640411583926, 'learning_rate': 0.010267436558567633, 'max_depth': 6, 'min_child_weight': 8.0, 'subsample': 0.989143389832629}


## Stacked ensemble model

In [21]:

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
from vecstack import stacking
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR

# Assuming train_df and val_df are defined elsewhere
X_train = np.array(train_df.drop('sellin', axis=1))
y_train = np.log1p(train_df["sellin"])
X_test = np.array(val_df.drop('sellin', axis=1))
y_test = np.log1p(val_df["sellin"])


# Models setup
models = [
  XGBRegressor(
    colsample_bytree=0.8031640411583926,
    learning_rate=0.010267436558567633,
    max_depth=6,
    min_child_weight=8.0,
    n_estimators=1000,
    subsample=0.989143389832629
),

  CatBoostRegressor(
    n_estimators=1500,
    border_count=3,
    depth=2,
    l2_leaf_reg=8.010730871768292,
    learning_rate=0.09969806104717538,
    verbose=0
),

  LGBMRegressor(
  n_estimators=2500,
  learning_rate=0.106882931763359658,
  max_depth=8,
  min_child_weight=2.5368947945207383,
  num_leaves=72

)
]

# Configure stacking
S_train, S_test = stacking(models,                   # list of models
                           X_train, y_train, X_test,    # data
                           regression=True,            # regression task (if classification, set to False)
                           mode='oof_pred_bag',        # method of aggregating the predictions
                           needs_proba=False,
                           save_dir=None,              # do not save result and log (change to your path if you need the logs)
                           metric=mean_squared_error,  # metric: callable
                           n_folds=10,                  # number of folds
                           stratified=False,
                           shuffle=False,               # shuffle the data
                           random_state=None,            # ensure reproducibility
                           verbose=2)

# Final model (meta-model)
meta_model = LGBMRegressor()

# Fit the meta-model
meta_model.fit(S_train, y_train)

# Make final predictions
final_predictions = np.expm1(meta_model.predict(S_test))
y_test_original = np.expm1(y_test)

# Evaluate final predictions
final_rmse = np.sqrt(mean_squared_error(y_test_original, final_predictions))
final_mae = mean_absolute_error(y_test_original, final_predictions)
final_mape = mean_absolute_percentage_error(y_test_original, final_predictions)
final_evs = explained_variance_score(y_test_original, final_predictions)

# Print final evaluation
print(f"Stacked Model Test RMSE: {final_rmse:.4f}")
print(f"Stacked Model Test MAE: {final_mae:.4f}")
print(f"Stacked Model Test MAPE: {final_mape:.4f}")
print(f"Stacked Model Test Explained Variance Score: {final_evs:.4f}")


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

LGBMRegressor()

Stacked Model Test RMSE: 65616.0951
Stacked Model Test MAE: 11188.8429
Stacked Model Test MAPE: 0.0512
Stacked Model Test Explained Variance Score: 0.9849
